## Working with custom layers

In this activity, we will take a look at how to create custom layers that allow you to not only display geo-spatial data but also animate your datapoints over time.  
We'll get a deeper understanding of how geoplotlib works and how layers are created and drawn.

Our dataset does not only contain spatial but also temporal information which enables us to plot flights over time on our map.   
There is an example on how to do this with taxis in the examples folder of geoplotlib.   
https://github.com/andrea-cuttone/geoplotlib/blob/master/examples/taxi.py

**Note:**   
The dataset can be found here:   
https://datamillnorth.org/dataset/flight-tracking

#### Loading the dataset

This time our dataset contains flight data recorded from different machines.   
Each entry is assigned to a unique plane through a `hex_ident`.   
Each location is related to a specific timestamp that consists of a `date` and a `time`.

In [1]:
# importing the necessary dependencies
import pandas as pd

In [2]:
# loading the dataset from the csv file
dataset = pd.read_csv('./data/flight_tracking.csv')

In [3]:
# displaying the first 5 rows of the dataset
dataset.head()

,hex_ident,altitude(feet),latitude,longitude,date,time,angle,distance(nauticalmile),squawk,ground_speed(knotph),track,callsign
0,40631C,14525,53.65947,-1.43819,2017/09/11,17:02:06.418,-120.77,11.27,6276.0,299.0,283.0,NaN
1,40631C,14525,53.65956,-1.43921,2017/09/11,17:02:06.875,-120.64,11.30,6276.0,299.0,283.0,NaN
2,40631C,14500,53.65979,-1.44066,2017/09/11,17:02:07.342,-120.43,11.32,6276.0,299.0,283.0,EZY63BT
3,40631C,14475,53.66025,-1.44447,2017/09/11,17:02:09.238,-119.94,11.40,6276.0,299.0,283.0,EZY63BT
4,40631C,14475,53.66044,-1.44591,2017/09/11,17:02:09.825,-119.75,11.43,6276.0,299.0,283.0,EZY63BT


In [4]:
# renaming columns latitude to lat and longitude to lon
dataset = dataset.rename(index=str, columns={"latitude": "lat", "longitude": "lon"})

**Note:**   
Remember that geoplotlib needs columns that are named `lat` and `lon`. You will encounter an error if that is not the case.

In [5]:
# displaying the first 5 rows of the dataset
dataset.head()

,hex_ident,altitude(feet),lat,lon,date,time,angle,distance(nauticalmile),squawk,ground_speed(knotph),track,callsign
0,40631C,14525,53.65947,-1.43819,2017/09/11,17:02:06.418,-120.77,11.27,6276.0,299.0,283.0,NaN
1,40631C,14525,53.65956,-1.43921,2017/09/11,17:02:06.875,-120.64,11.30,6276.0,299.0,283.0,NaN
2,40631C,14500,53.65979,-1.44066,2017/09/11,17:02:07.342,-120.43,11.32,6276.0,299.0,283.0,EZY63BT
3,40631C,14475,53.66025,-1.44447,2017/09/11,17:02:09.238,-119.94,11.40,6276.0,299.0,283.0,EZY63BT
4,40631C,14475,53.66044,-1.44591,2017/09/11,17:02:09.825,-119.75,11.43,6276.0,299.0,283.0,EZY63BT


---

#### Adding an unix timestamp

The easiest way to work with and handle time is to use a unix timestamp.   
In previous activities, we've already seen how to create a new column in our dataset by applying a function to it.   
We are using the datatime library to parse the date and time columns of our dataset and use it to create a unix timestamp.

In [6]:
# method to convert date and time to an unix timestamp
from datetime import datetime

def to_epoch(date, time):
    try:
        timestamp = round(datetime.strptime('{} {}'.format(date, time), '%Y/%m/%d %H:%M:%S.%f').timestamp())
        return timestamp
    except ValueError:
        return round(datetime.strptime('2017/09/11 17:02:06.418', '%Y/%m/%d %H:%M:%S.%f').timestamp())

In [7]:
# creating a new column called timestamp with the to_epoch method applied
dataset['timestamp'] = dataset.apply(lambda x: to_epoch(x['date'], x['time']), axis=1)

In [8]:
# displaying the first 5 rows of the dataset
dataset.head()

,hex_ident,altitude(feet),lat,lon,date,time,angle,distance(nauticalmile),squawk,ground_speed(knotph),track,callsign,timestamp
0,40631C,14525,53.65947,-1.43819,2017/09/11,17:02:06.418,-120.77,11.27,6276.0,299.0,283.0,NaN,1505142126
1,40631C,14525,53.65956,-1.43921,2017/09/11,17:02:06.875,-120.64,11.30,6276.0,299.0,283.0,NaN,1505142127
2,40631C,14500,53.65979,-1.44066,2017/09/11,17:02:07.342,-120.43,11.32,6276.0,299.0,283.0,EZY63BT,1505142127
3,40631C,14475,53.66025,-1.44447,2017/09/11,17:02:09.238,-119.94,11.40,6276.0,299.0,283.0,EZY63BT,1505142129
4,40631C,14475,53.66044,-1.44591,2017/09/11,17:02:09.825,-119.75,11.43,6276.0,299.0,283.0,EZY63BT,1505142130


**Note:**   
We round up the miliseconds in our `to_epoch` method since epoch is the number of seconds (not miliseconds) that have passes since January 1st 1970.   
Of course we loose some precision here, but we want to focus on creating our own custom layer instead of wasting a lot of time with our dataset.

---

#### Writing our custom layer

After preparing our dataset, we can now start writing our custom layer.   
As mentioned at the beginning of this activity, it will be based on the taxi example of geoplotlib.   

We want to have a layer `TrackLayer` that takes an argument dataset which contains `lat` and `lon` data in combination with a `timestamp`.   
Given this data, we want to plot each point for each timestamp on the map, creating a tail behind the newest position of the plane.
The geoplotlib colorbrewer is used to give each plane a color based on their unique `hex_ident`.   
The view (bounding box) of our visualization will be set to the city Leeds and a text information with the current timestamp is displayed in the upper right corner.

In [9]:
# custom layer creation
import geoplotlib
from geoplotlib.layers import BaseLayer
from geoplotlib.core import BatchPainter
from geoplotlib.colors import colorbrewer
from geoplotlib.utils import epoch_to_str, BoundingBox

class TrackLayer(BaseLayer):

    def __init__(self, dataset, bbox=BoundingBox.WORLD):
        self.data = dataset
        self.cmap = colorbrewer(self.data['hex_ident'], alpha=200)
        self.time = self.data['timestamp'].min()
        self.painter = BatchPainter()
        self.view = bbox


    def draw(self, proj, mouse_x, mouse_y, ui_manager):
        self.painter = BatchPainter()
        df = self.data.where((self.data['timestamp'] > self.time) & (self.data['timestamp'] <= self.time + 180))

        for element in set(df['hex_ident']):
            grp = df.where(df['hex_ident'] == element)
            self.painter.set_color(self.cmap[element])
            x, y = proj.lonlat_to_screen(grp['lon'], grp['lat'])
            self.painter.points(x, y, 15, rounded=True)

        self.time += 1

        if self.time > self.data['timestamp'].max():
            self.time = self.data['timestamp'].min()

        self.painter.batch_draw()
        ui_manager.info('Current timestamp: {}'.format(epoch_to_str(self.time)))
        
    # bounding box that gets used when layer is created
    def bbox(self):
        return self.view

---

#### Visualization with of the custom layer

After creating the custom layer, using it is as simple as using any other layer in geoplotlib.   
We can use the `add_layer` method and pass in our custom layer class with the parameters needed.

Our data is focused on the UK and specifically Leeds.   
So we want to adjust our bounding box to exactly this area.

In [10]:
# bounding box for our view on leeds
from geoplotlib.utils import BoundingBox

leeds_bbox = BoundingBox(north=53.8074, west=-3, south=53.7074 , east=0)

In [11]:
# displaying our custom layer using add_layer
from geoplotlib.utils import DataAccessObject

data = DataAccessObject(dataset)

geoplotlib.add_layer(TrackLayer(data, bbox=leeds_bbox))
geoplotlib.show()

**Note:**   
In order to avoid any errors associated with the library, we have to convert our pandas dataframe to a geoplotlib DataAccessObject.   
The creator of geoplotlib provides a handy interface for this conversion.

When looking at the upper right hand corner, we can clearly see the temporal aspect of this visualization.   
The first observation we make is that our data is really sparse, we sometimes only have a single data point for a plane, seldomly a whole path is drawn.   

Even though it is so sparse, we can already get a feeling about where the planes are flying most.

**Note:**   
If you're interested in what else can be achieved with this custom layer approach, there are more examples in the geoplotlib repository.   
- https://github.com/andrea-cuttone/geoplotlib/blob/master/examples/follow_camera.py
- https://github.com/andrea-cuttone/geoplotlib/blob/master/examples/quadtree.py
- https://github.com/andrea-cuttone/geoplotlib/blob/master/examples/kmeans.py